# Association Rules

In [3]:
#import libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from pyclustering.cluster.kmedoids import kmedoids
from yellowbrick.cluster import KElbowVisualizer

In [2]:
#data should have transactions + items
o_items = pd.read_csv("C:/Users/TASNIM/Desktop/LUISS/NTT/Datasets/05.order_items.csv", encoding= 'unicode_escape', sep =";")
products = pd.read_csv("C:/Users/TASNIM/Desktop/LUISS/NTT/Datasets/08.products.csv", encoding= 'unicode_escape', sep =";")

In [3]:
o_items.isna().sum()

order_id                    5
order_item_sequence_id      0
product_id                  0
price                       0
shipping_cost               0
seller_id                   0
max_shipping_seller_date    0
dtype: int64

In [4]:
products.isna().sum()

product_id                  0
product_category_name     610
product_weight_gr           2
product_length_cm           2
product_height_cm           2
product_width_cm            2
product_photo_quantity    610
dtype: int64

In [5]:
#drop order ids with NAs
o_items = o_items.dropna()

In [6]:
#fill nas with Other category
products.product_category_name = products.product_category_name.fillna("Other")

In [7]:
#only keep category name and id for merging
products = products[["product_id", "product_category_name"]]

In [8]:
#add the category to our o_items
o_items = o_items.merge(products, on = "product_id")

In [9]:
#we first converrt the variables to their appropriate type
#we convert the shipping cost to float type
o_items.shipping_cost= o_items.shipping_cost.str.replace(',', '.')
o_items.shipping_cost = o_items.shipping_cost.astype('float')

In [10]:
#we create a loop to remove the first comma in our dataset, and replace the second with a dot
pd.options.mode.chained_assignment = None  # default='warn'
for i in range(len(o_items)):
    if len(o_items.price[i]) > 8:
        o_items.price[i] = list(o_items.price[i])
        del o_items.price[i][1]
        o_items.price[i] = ''.join(o_items.price[i])

In [11]:
o_items.product_category_name.unique()

array(['cell phones', 'beauty & personal care', 'handbags & accessories',
       'furniture', 'automotive', 'comics', 'toys games',
       'computer accessories', 'sport outdoors', 'kitchen & dining',
       'home audio', 'Other', 'kids', 'videogame console', 'lawn garden',
       'fragrance', 'home appliances', 'office products', 'fire safety',
       'home accessories', "men's fashion", "woman's fashion",
       'pet supplies', 'television & video', 'fashion & shoes',
       'business office', 'headphones', 'jewelry', 'cleaning supplies',
       'book', 'light bulbs', 'pet food', 'bedroom decor', 'cd vinyl',
       'home lighting', 'monitors', 'underwear', 'agriculture suppliers',
       'watches', 'safety apparel', 'luggage', 'sofa',
       'music instruments', 'computers tablets', 'hardware', 'dvd',
       'film & photography', 'wellness & relaxation',
       'tools home improvement', 'health household', 'ceiling fans',
       'painting', 'home emergency kits', 'oral care', 'fabric

In [12]:
#now we convert to float & str
o_items.price= o_items.price.str.replace(',', '.')
o_items.price = o_items.price.astype('float')
o_items.product_category_name = o_items.product_category_name.astype("str")

In [13]:
o_items['order_item_sequence_id'] = o_items['order_item_sequence_id'].astype(str)
o_items['product_id'] = o_items['product_id'].astype(str)
#create a dataset for our association rules analysis
b = o_items.groupby(['order_id'], as_index = False).agg({'product_category_name': lambda x: ','.join(x.unique()), 'order_item_sequence_id': 'max', 'product_id': lambda x: ','.join(x.unique()), 'price': 'sum', 'shipping_cost': 'sum'})

## Association Rules Per Product

In [14]:
#keep only the transactions that bought two different products
ar_product = b[b['product_id'].str.contains(",")]

In [15]:
ar_product.shape #we have 3236 transactions that bought two different products together

(3236, 6)

In [16]:
#Let’s split the products and create a list called by ‘data’,
data = list(ar_product["product_id"].apply(lambda x:x.split(",") ))

In [17]:
#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df

,0011c512eb256aa0dbbb544d8dffcf6e,001b72dfd63e9833e8c02742adf472e3,0042f1a9a7e0edd1400c6cd0fda065f8,005030ef108f58b46b78116f754d8d38,0060b415594c5e1200324ef1a18493c4,007c63ae4b346920756b5adcad8095de,008cff0e5792219fae03e570f980b330,00ae7076313576f94d9107599d79a978,00ba6d766f0b1d7b78a5ce3e1e033263,00d2fa6f2a87633084c8e0400d41a65a,...,ff85ff517698c3fe8b200afddda7fb3a,ff9032482857269e8683f0c2f8ba4709,ff95ac47246ef13e48712ea1ff8df0d9,ffb2e8c1ddc7c3e590d2bc4c91de53e1,ffbc83054b3741a8d67fc59d9cf9d42d,ffcfaba393e8ef71937c6e8421bc2868,ffd4bf4306745865e5692f69bd237893,ffe8083298f95571b4a66bfbc1c05524,fff1059cd247279f3726b7696c66e44e,fff28f91211774864a1000f918ed00cc
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
from mlxtend.frequent_patterns import apriori, association_rules
d = apriori(df, min_support = 0.01, use_colnames = True, verbose = 1)
d

Processing 12 combinations | Sampling itemset size 2


,support,itemsets
0,0.011125,(35afc973633aaeb6b877ff57b2793310)
1,0.014833,(36f60d45225e60c7da4558b070ce4b60)
2,0.016069,(99a4788cb24856965c36a24e339b6058)
3,0.011743,(e53e557d5a159f5aa2c5e995dfdf244b)
4,0.010507,"(e53e557d5a159f5aa2c5e995dfdf244b, 36f60d45225..."


In [19]:
#Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(d, min_threshold = 0)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(e53e557d5a159f5aa2c5e995dfdf244b),(36f60d45225e60c7da4558b070ce4b60),0.011743,0.014833,0.010507,0.894737,60.320175,0.010333,9.359085
1,(36f60d45225e60c7da4558b070ce4b60),(e53e557d5a159f5aa2c5e995dfdf244b),0.014833,0.011743,0.010507,0.708333,60.320175,0.010333,3.388310


## Association Rules Per Category

In [20]:
#keep only the transactions that bought two different categories
ar_cat = b[b['product_category_name'].str.contains(",")]

In [21]:
ar_cat.shape #we have 783 transactions that bought two different products together

(783, 6)

In [32]:
ar_cat.head()

,order_id,product_category_name,order_item_sequence_id,product_id,price,shipping_cost
73,002f98c0f7efd42638ed6100ca699b42,"toys games,videogame console",2,"880be32f4db1d9f6e2bec38fb6ac23ab,d41dc2f2979f5...",53.89,39.73
134,005d9a5423d47281ac463a968b3936fb,"kids,toys games",3,"4c3ae5db49258df0784827bdacf3b396,fb7a100ec8c7b...",99.97,45.28
485,014405982914c2cde2796ddcf0b8703d,"fragrance,sport outdoors",2,"6782d593f63105318f46bbf7633279bf,e95ee6822b66a...",49.23,29.20
637,01b1a7fdae9ad1837d6ab861705a1fa5,"kitchen & dining,home accessories",2,"3fae92f8d0ebb3317991934a6d717c47,9b02b650be0a3...",108.99,26.85
688,01cce1175ac3c4a450e3a0f856d02734,"book,lawn garden",2,"9d0aa87e8df1bdbe0f79353520a2d538,415dfa57292b8...",96.23,27.51


In [22]:
#Let’s split the products and create a list called by ‘data’,
data = list(ar_cat["product_category_name"].apply(lambda x:x.split(",") ))
data

[['toys games', 'videogame console'],
 ['kids', 'toys games'],
 ['fragrance', 'sport outdoors'],
 ['kitchen & dining', 'home accessories'],
 ['book', 'lawn garden'],
 ['fire safety', 'beauty & personal care'],
 ['computer accessories', 'book'],
 ['sport outdoors', 'comics'],
 ['home accessories', 'health household'],
 ['comics', 'kids'],
 ['computer accessories', 'cell phones'],
 ['furniture', 'music instruments'],
 ['beauty & personal care', 'home accessories'],
 ['furniture', 'tools home improvement'],
 ['food', 'home accessories'],
 ['book', 'luggage'],
 ['comics', 'cell phones'],
 ['sport outdoors', 'fragrance'],
 ['sport outdoors', 'home accessories'],
 ['tools home improvement', 'kitchen & dining'],
 ['beauty & personal care', 'kids'],
 ['headphones', 'jewelry'],
 ['furniture', 'tools home improvement'],
 ['kitchen & dining', 'furniture'],
 ['furniture', 'wellness & relaxation'],
 ['toys games', 'fragrance'],
 ['kitchen & dining', 'furniture'],
 ['home accessories', 'lawn garden'

In [38]:
#Let's transform the list, with one-hot encoding
from mlxtend.preprocessing import TransactionEncoder
a = TransactionEncoder()
a_data = a.fit(data).transform(data)
df = pd.DataFrame(a_data,columns=a.columns_)
df = df.replace(False,0)
df.head()

,Other,automotive,beauty & personal care,bedroom decor,book,business office,cd vinyl,cell phones,coffee machines,comics,...,sport outdoors,television & video,tools home improvement,toys games,videogame,videogame console,wall art,watches,wellness & relaxation,woman's fashion
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,True,0,True,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,True,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,True,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,True,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from mlxtend.frequent_patterns import apriori, association_rules
d = apriori(df, min_support = 0.01, use_colnames = True, verbose = 1)
d

Processing 294 combinations | Sampling itemset size 3


,support,itemsets
0,0.079183,(Other)
1,0.045977,(automotive)
2,0.091954,(beauty & personal care)
3,0.048531,(book)
4,0.034483,(cell phones)
5,0.010217,(coffee machines)
6,0.085568,(comics)
7,0.066411,(computer accessories)
8,0.025543,(fashion & shoes)
9,0.015326,(fire safety)


In [25]:
#Let's view our interpretation values using the Associan rule function.
df_ar = association_rules(d, metric = "confidence", min_threshold = 0.2)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(fashion & shoes),(Other),0.025543,0.079183,0.011494,0.450000,5.683065,0.009472,1.674213
1,(Other),(home accessories),0.079183,0.148148,0.017880,0.225806,1.524194,0.006149,1.100309
2,(fragrance),(beauty & personal care),0.035760,0.091954,0.015326,0.428571,4.660714,0.012037,1.589080
3,(kids),(comics),0.118774,0.085568,0.025543,0.215054,2.513240,0.015379,1.164961
4,(comics),(kids),0.085568,0.118774,0.025543,0.298507,2.513240,0.015379,1.256216
5,(home accessories),(furniture),0.148148,0.263091,0.030651,0.206897,0.786408,-0.008325,0.929147
6,(furniture),(kitchen & dining),0.263091,0.260536,0.089400,0.339806,1.304255,0.020855,1.120070
7,(kitchen & dining),(furniture),0.260536,0.263091,0.089400,0.343137,1.304255,0.020855,1.121862
8,(lawn garden),(furniture),0.093231,0.263091,0.021711,0.232877,0.885158,-0.002817,0.960614
9,(tools home improvement),(furniture),0.063857,0.263091,0.030651,0.480000,1.824466,0.013851,1.417133


In [11]:
df = pd.read_csv('C:/Users/TASNIM/Downloads/df_with_clusters.csv', index_col = 0)

In [12]:
df

,Segment K-means PCA,customer_unique_id,new_diff,customer_autonomous_community,customer_city,payment_method,transaction_value,payment_method_sequence_id,og_cat_combined,new_cat_combined,tot_spending,max_shipping,tot_order_item_count,installment_payment,del/rev_mean,rev_count/tot_items_mean,Frequency,tenure,Recency,tot_ship/tot_spend
ID,,,,,,,,,,,,,,,,,,,,
0,3,871766c5855e863f6eccc05f988b23cb,15,cataluna,tarragona,credit_card,72.20,1.0,comics,books,72.20,13.29,1,1,1.0,1.0,1,1,351,0.1841
1,2,eb28e67c4c0b83846050ddfb8a35d051,18,andalucia,cadiz,credit_card,284.50,1.0,"pet supplies,computer accessories","pets,computers & phones",284.50,19.94,2,1,2.5,1.0,2,146,345,0.1117
2,3,af861d436cfc08b2c2ddefd0ba074622,11,andalucia,cadiz,credit_card,25.78,1.0,fragrance,beauty & personal care,25.78,12.79,1,1,1.0,1.0,1,1,22,0.4960
3,3,64b576fb70d441e8f1b2d7d446e483c5,40,andalucia,cordoba,credit_card,218.00,1.0,lawn garden,garden,218.00,18.14,1,1,1.0,1.0,1,1,572,0.0832
4,3,635d9ac1680f03288e72ada3a1035803,24,andalucia,cadiz,credit_card,31.75,1.0,cell phones,computers & phones,31.75,11.85,1,0,10.0,1.0,1,1,263,0.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90746,3,ecc3d4eb9b17d2f0865d21f2abecc51c,33,comunidad de madrid,getafe,credit_card,139.90,1.0,tools home improvement,home & kitchen,139.90,20.03,1,1,3.0,1.0,1,1,72,0.1432
90747,1,0da9fe112eae0c74d3ba1fe16de0988b,17,galicia,coruna,cash,386.50,1.0,computer accessories,computers & phones,386.50,36.53,1,0,1.0,1.0,1,1,47,0.0945
90748,3,cd79b407828f02fdbba457111c38e4c4,17,comunidad de madrid,madrid,credit_card,116.90,1.0,sport outdoors,sport,116.90,16.95,1,1,1.0,1.0,1,1,310,0.1450


In [30]:
df

,Segment K-means PCA,customer_unique_id,new_diff,customer_autonomous_community,customer_city,payment_method,transaction_value,payment_method_sequence_id,og_cat_combined,new_cat_combined,tot_spending,max_shipping,tot_order_item_count,installment_payment,del/rev_mean,rev_count/tot_items_mean,Frequency,tenure,Recency,tot_ship/tot_spend
ID,,,,,,,,,,,,,,,,,,,,
0,3,871766c5855e863f6eccc05f988b23cb,15,cataluna,tarragona,credit_card,72.20,1.0,comics,books,72.20,13.29,1,1,1.0,1.0,1,1,351,0.1841
1,2,eb28e67c4c0b83846050ddfb8a35d051,18,andalucia,cadiz,credit_card,284.50,1.0,"pet supplies,computer accessories","pets,computers & phones",284.50,19.94,2,1,2.5,1.0,2,146,345,0.1117
2,3,af861d436cfc08b2c2ddefd0ba074622,11,andalucia,cadiz,credit_card,25.78,1.0,fragrance,beauty & personal care,25.78,12.79,1,1,1.0,1.0,1,1,22,0.4960
3,3,64b576fb70d441e8f1b2d7d446e483c5,40,andalucia,cordoba,credit_card,218.00,1.0,lawn garden,garden,218.00,18.14,1,1,1.0,1.0,1,1,572,0.0832
4,3,635d9ac1680f03288e72ada3a1035803,24,andalucia,cadiz,credit_card,31.75,1.0,cell phones,computers & phones,31.75,11.85,1,0,10.0,1.0,1,1,263,0.3733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90746,3,ecc3d4eb9b17d2f0865d21f2abecc51c,33,comunidad de madrid,getafe,credit_card,139.90,1.0,tools home improvement,home & kitchen,139.90,20.03,1,1,3.0,1.0,1,1,72,0.1432
90747,1,0da9fe112eae0c74d3ba1fe16de0988b,17,galicia,coruna,cash,386.50,1.0,computer accessories,computers & phones,386.50,36.53,1,0,1.0,1.0,1,1,47,0.0945
90748,3,cd79b407828f02fdbba457111c38e4c4,17,comunidad de madrid,madrid,credit_card,116.90,1.0,sport outdoors,sport,116.90,16.95,1,1,1.0,1.0,1,1,310,0.1450


In [36]:
# new data frame with split value columns 
df = df.assign(new_cat_combined=df['new_cat_combined'].str.split(',')).explode('new_cat_combined')

In [39]:
df.to_csv("clusters_cat.csv")